In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "ProgressMeter",
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
modified_fasta_files = sort(filter(x -> occursin(r"\.fna$", x) && occursin(".vcf", x) && occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
for fasta_file in modified_fasta_files
    ProgressMeter.@showprogress for coverage in [2^i for i in 0:10]
        outprefix = "$(fasta_file).art.$(coverage)x."
        forward = outprefix * "1.fq.gz"
        reverse = outprefix * "2.fq.gz"
        if isfile(forward) && isfile(reverse) && (filesize(forward) > 0) && (filesize(reverse) > 0)
            # @info "$(forward) and $(reverse) found, skipping..."
            continue
        else
            Mycelia.simulate_short_reads(in_fasta = fasta_file, coverage = coverage)
        end
    end
end